In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Reading Data


In [2]:
df_2021 = pd.read_csv('data/2021.csv')
df_2021.dropna(inplace=True)
df_2021.tail(20)

,Plot,Treatment,BD 0-10cm,BD 10-20cm,BD 20-30cm,BD Moisture 0-10cm,BD Moisture 10-20 cm,BD Moisture 20-30 cm,Emergence,Chlorophyll 28th May,...,Cone P NT 25cm,Cone P NT 27.5cm,Cone P NT 30cm,Cone P NT 32.5cm,Cone P NT 35cm,Cone P NT 37.5cm,Cone P NT 40cm,Cone P NT 42.5cm,Cone P NT 45cm,Combine Yield (t/ac)
39,40,PLPHMNCLA,1.19,1.26,1.51,19.2,30.1,29.2,9.6,39.2,...,1096.0,1914.0,2693.0,1776.0,3561.0,4448.0,4110.0,3889.0,3453.0,2.94
40,41,PHPLMCCLA,1.28,1.43,1.39,18.7,29.3,31.9,9.2,42.9,...,1885.0,1951.0,1913.0,1745.0,2084.0,2278.0,3579.0,3627.0,1978.0,2.96
41,42,PLPLMNCHA,1.12,1.26,1.43,17.8,30.5,29.4,9.2,41.3,...,1016.0,1163.0,1365.0,1417.0,2071.0,3708.0,4054.0,4196.0,3808.0,2.69
43,44,PLPLMNCLA,1.13,1.28,1.30,19.2,34.4,33.0,9.2,39.2,...,1925.0,2533.0,2976.0,3230.0,2908.0,3917.0,4514.0,3991.0,4055.0,2.53
45,46,PHPLMNCLA,1.18,1.43,1.65,19.4,28.6,30.8,9.2,43.3,...,552.0,560.0,835.0,1336.0,2076.0,1632.0,3019.0,1392.0,2006.0,2.36
57,58,MLPHMNCLA,1.18,1.61,1.67,15.4,25.4,22.4,7.6,40.1,...,4883.0,4475.0,3962.0,2360.0,4142.0,4349.0,2499.0,1392.0,3207.0,2.13
59,60,MHPHMNCLA,1.22,1.66,1.62,16.7,20.4,21.1,6.4,39.1,...,2755.0,2246.0,3376.0,4152.0,4604.0,4166.0,4571.0,4299.0,4446.0,2.02
64,65,PLPHMCCLA,1.25,1.29,1.47,15.3,18.6,22.3,6.2,38.4,...,1903.0,1780.0,109.0,2840.0,3010.0,3714.0,3814.0,4045.0,3495.0,1.77
65,66,PLPLMCCHA,1.25,1.58,1.70,13.6,25.3,18.4,7.8,39.3,...,2020.0,2109.0,2411.0,2664.0,2902.0,3670.0,3551.0,4039.0,3056.0,1.96
66,67,PLPHMNCHA,1.36,1.40,1.64,17.3,21.7,19.6,7.2,43.6,...,1122.0,1270.0,1805.0,2322.0,3023.0,4384.0,4652.0,4544.0,4360.0,2.29


### Setting up data for models & testing

In [4]:
# Setting targets and removing them from dataframe
y = df_2021['Combine Yield (t/ac)']
del df_2021['Combine Yield (t/ac)']

emergenceY = df_2021['Emergence']
del df_2021['Emergence']

# Features
X = df_2021[['BD Moisture 0-10cm', 'BD Moisture 10-20 cm', 'BD Moisture 20-30 cm']]

In [5]:
# Splitting data for training & testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

X_train_emergence, X_test_emergence, y_train_emergence, y_test_emergence = train_test_split(X, emergenceY, test_size=0.33, random_state=42) # Emergence

## Linear Regression Model

In [6]:
# Linear Regression Model
reg = LinearRegression().fit(X_train, y_train)
reg.score(X, y) # score

-0.027511932014425033

In [7]:
# Linear Regression Model
reg = LinearRegression().fit(X_train_emergence, y_train_emergence)
reg.score(X, emergenceY) # score

-0.01060712382413187

In [16]:
reg.predict(np.array([[17.7, 31.9, 29.4]]))

/Users/ciaranflanagan/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([7.1702804])

## SVR Model

In [9]:
# SVR Model
svr = SVR().fit(X_train, y_train)
svr.score(X, y) # score

0.0996501249929731

In [10]:
# SVR Model
svr = SVR().fit(X_train_emergence, y_train_emergence)
svr.score(X, emergenceY) # score

0.16962123214611857

In [15]:
svr.predict(np.array([[17.7, 31.9, 29.4]]))

/Users/ciaranflanagan/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(


array([7.74073045])

In [12]:
# Starting point -> improved models
# target = emergence date / emergence count